# Accessing OpenAI through Vocareum API keys

In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
os.getenv("OPENAI_API_KEY")

'voc-8957811371266772085699678d66e1475c97.62448043'

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="voc-8957811371266772085699678d66e1475c97.62448043",
    base_url="https://openai.vocareum.com/v1"
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

print(completion.choices[0].message.content)

Arrr, to be seein' if a Python object be an instance of a particular class, ye use the trusty `isinstance()` function. Here be how ye do it:

```python
if isinstance(your_object, YourClass):
    print("Aye, the object be an instance of the class!")
else:
    print("Nay, the object be not of that class!")
```

Just replace `your_object` with the object ye be checkin', and `YourClass` with the name of the class ye be inspectin'. Arrr, that be the way to find the truth on the high seas of code!


In [4]:
from openai import OpenAI
client = OpenAI(
    base_url = openai.api_base,
    api_key = openai.api_key
)

In [4]:
def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


# Extracting data from Wikipedia

## Getting a list of all pages from a Wikipedia category

In [13]:
import wikipediaapi
import openai
import pandas as pd
import json
import os

In [17]:
def fetch_wikipedia_page(page_title):
    page = wiki_wiki.page(page_title)
    if page.exists():
        return page.text
    else:
        return None

In [18]:
wiki_wiki = wikipediaapi.Wikipedia('CustomChatbot (gabrielgoncalvesbr@gmail.com)', 'en')

In [19]:
if os.path.exists("nba_playoffs_wikipedia.json"):
    # Read the JSON file and load it into the dictionary
    with open("nba_playoffs_wikipedia.json", "r", encoding="utf-8") as json_file:
        dict_nba_playoffs_pages = json.load(json_file)
else:
    dict_nba_playoffs_pages = {}
    
    cat = wiki_wiki.page("Category:NBA playoffs")
    for title in cat.categorymembers:
        print(title)
        page_result = fetch_wikipedia_page(title)
        if page_result:
            dict_nba_playoffs_pages[title] = page_result

    with open("nba_playoffs_wikipedia.json", "w", encoding="utf-8") as json_file:
        json.dump(dict_nba_playoffs_pages, json_file, ensure_ascii=False, indent=4)


# Preparing the dataset

In [20]:
df_playoffs = pd.DataFrame.from_dict(
    dict_nba_playoffs_pages, 
    orient='index', 
    columns=['text']#, 'text']
).reset_index().rename(columns={'index': 'title'})
df_playoffs.head()

,title,text
0,NBA playoffs,The NBA playoffs is the annual elimination tou...
1,1950 NBA playoffs,The 1950 NBA playoffs was the postseason tourn...
2,1951 NBA playoffs,The 1951 NBA playoffs was the postseason tourn...
3,1952 NBA playoffs,The 1952 NBA playoffs was the postseason tourn...
4,1953 NBA playoffs,The 1953 NBA playoffs was the postseason tourn...


In [21]:
df_playoffs[df_playoffs['title'].str.contains("2023")]

,title,text
74,2023 NBA playoffs,The 2023 NBA playoffs was the postseason tourn...


# Naive OpenAI NBA Playoffs Chatbot
In this session, we are going to built a naive OpenAI chatbot, that captures the information about the year and a question about NBA playoffs and makes a request to OpenAI. 

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openai.vocareum.com/v1"
)

def chat_with_openai(messages):
    """
    Sends a conversation to OpenAI and gets a response using the updated API.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Replace with "gpt-4" or "gpt-3.5-turbo" as needed
            messages=messages
        )
        # Access the content of the response using dot notation
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def naive_OpenAI_chatbot():
    """
    Starts the chatbot interaction.
    """
    print("👋 Welcome to the NBA Playoffs Chatbot!")
    
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": "You are a helpful assistant knowledgeable about NBA playoffs."}
    ]
    
    # Ask the first question
    year = input("Bot: Which NBA playoff year do you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about the NBA playoffs in {year}."})
    
    # Ask the second question
    info_type = input("Bot: What's the information you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about {info_type} in the {year} NBA playoffs."})
    
    # Generate a response
    print("Bot: Let me gather some information for you...\n")
    
    assistant_prompt = f"Generate a detailed paragraph about {info_type} in the {year} NBA playoffs."
    conversation.append({"role": "assistant", "content": assistant_prompt})
    
    response = chat_with_openai(conversation)
    
    if response:
        # Display the generated paragraph
        print(f"Bot: Here's what I found:\n{response}\n")
        
        # Confirm with the user
        is_correct = input("Bot: Is this what you were searching for? (yes/no)\nYou: ").strip().lower()
        
        if is_correct == 'yes':
            print("Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.")
        else:
            print("Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?")
            naive_OpenAI_chatbot()
    else:
        print("Bot: I couldn't retrieve any information. Please try again later.")


## Asking questions to the Naive Chatbot
To test the Naive Chatbot, we are going to ask the following questions:

1. Question 1<br>
    - Year: **2020**
    - Question: **What was the starting five for the two teams in the NBA finals, each player basic statistics?**

2. Question 2
    - Year: **2022**
    - Question: **Which team was the champion and which player was named finals MVP?**

### Question 1 (Naive Chatbot)

In [ ]:
naive_OpenAI_chatbot()

### Question 2 (Naive Chatbot)

In [9]:
naive_OpenAI_chatbot()

👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2022
Bot: What's the information you want to know about?
You:  Which team was the champion and which player was named finals MVP


Bot: Let me gather some information for you...

Bot: Here's what I found:
I'm sorry, but as an AI developed by OpenAI, I can't provide real-time updates or future information. As of the last update in October 2021, the 2022 NBA playoffs hadn't taken place yet, and so the champions and Finals MVP haven't been determined. Please check the most recent and reliable sports news sources for this information.



Bot: Is this what you were searching for? (yes/no)
You:  no


Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?
👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  no
Bot: What's the information you want to know about?
You:  no


Bot: Let me gather some information for you...

Bot: Here's what I found:
I'm sorry, but your question is unclear. Could you please clarify what specific information about NBA playoffs you are interested in? Maybe you're asking about a specific year for the NBA playoffs or a specific team or player? Please give more details so I can assist you better.



Bot: Is this what you were searching for? (yes/no)
You:  yes


Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.


In [ ]:
df_playoffs[df_playoffs['title'].str.contains("2022")]['text'].str.cat(sep='\n')  #

In [34]:
def get_data_from_nba_playoffs(
    query: str, dataset: pd.DataFrame = df_playoffs
) -> str:
    cat_text = dataset[
        dataset["title"].str.contains(str(query))]["text"].str.cat(sep="\n")
    return cat_text

In [40]:
get_data_from_nba_playoffs(1789)

''

In [46]:
def augmented_openai_chatbot():
    print("👋 Welcome to the Augmented NBA Playoffs Chatbot!")
    
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": "You are a helpful assistant knowledgeable about NBA playoffs."}
    ]
    
    # Ask the first question
    year = input("Bot: Which NBA playoff year do you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about the NBA playoffs in {year}."})
    
    # Ask the second question
    info_type = input("Bot: What's the information you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about {info_type} in the {year} NBA playoffs."})
    
    # Generate a response
    print("Bot: Let me gather some information for you...\n")

    # Get Data from Wikipedia dataset
    str_filtered_dataset = get_data_from_nba_playoffs(year)
    if str_filtered_dataset != '':
        print(f'''Found information about year "{year}" in NBA playoffs,
        with {len(str_filtered_dataset)} characters.
        ''')
    
    assistant_prompt = f"Generate a detailed paragraph about {info_type} in the {year} NBA playoffs.\n"
    augemented_prompt = assistant_prompt + f"Also use the following wikipedia paragraph as a source of information, if available:\n{str_filtered_dataset}"
    conversation.append({"role": "assistant", "content": augemented_prompt})
    
    response = chat_with_openai(conversation)
    
    if response:
        # Display the generated paragraph
        print(f"Bot: Here's what I found:\n{response}\n")
        
        # Confirm with the user
        is_correct = input("Bot: Is this what you were searching for? (yes/no)\nYou: ").strip().lower()
        
        if is_correct == 'yes':
            print("Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.")
        else:
            print("Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?")
            naive_OpenAI_chatbot()
    else:
        print("Bot: I couldn't retrieve any information. Please try again later.")


In [ ]:
 - Year: **2022**
    - Question: **Which team was the champion and which player was named finals MVP?**

In [48]:
augmented_openai_chatbot()

👋 Welcome to the Augmented NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2022
Bot: What's the information you want to know about?
You:  Which team was the champion and which player was named finals MVP?


Bot: Let me gather some information for you...

Found information about year "2022" in NBA playoffs,
        with 55284 characters.
        
An error occurred: 'NoneType' object is not subscriptable
Bot: I couldn't retrieve any information. Please try again later.
